In [60]:
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio import SeqIO

In [61]:
def calculate_features(seq):
    # Calculate base frequencies
    num_bases = len(seq)
    freq_a = seq.count('A') / num_bases
    freq_c = seq.count('C') / num_bases
    freq_g = seq.count('G') / num_bases
    freq_t = seq.count('T') / num_bases
    freq_t = 1 if freq_t==0 else freq_t
    
    # Calculate fickett score
    r_y_ratio = (freq_a + freq_g) / (freq_c + freq_t)
    a_t_ratio = freq_a / freq_t
    gc_content = freq_g + freq_c
    fickett_score = (r_y_ratio * a_t_ratio) + gc_content - 0.5
    
    # Calculate CpG island features
    cpg_islands , cpg_count = [] , 0
    
    for i in range(len(seq)-1):
        if seq[i:i+2].upper() == 'CG':
            cpg_count += 1
            if cpg_count == 1:
                cpg_islands.append(0)
        else:
            if cpg_count > 0:
                cpg_islands[-1] += 1
                cpg_count = 0
    if cpg_count > 0:
        cpg_islands[-1] += 1
    num_islands = len(cpg_islands)
    
     # Calculate GC count
    gc_count = ((seq.count('G') + seq.count('C')) / num_bases) * 100
    
    # Calculate longest ORF length
    longest_orf_length = 0
    for frame in range(3):
        for pos in range(frame, len(seq)-2, 3):
            codon = seq[pos:pos+3]
            if codon == 'ATG':
                orf_length = 0
                for pos2 in range(pos, len(seq)-2, 3):
                    codon2 = seq[pos2:pos2+3]
                    if codon2 in ('TAA', 'TAG', 'TGA'):
                        break
                    else:
                        orf_length += 3
                longest_orf_length = max(longest_orf_length, orf_length)
    
   

    # Calculate transcript length
    transcript_length = len(seq)

    # Calculate ORF ratio
    orf_ratio = longest_orf_length / transcript_length
    
    # Calculate AT dinucleotide
    dinucs = ["AT"]
    dinuc_counts = [0]*len(dinucs)
    seq_len = len(seq)
    
    for i in range(seq_len-1):
        dinuc = seq[i:i+2]
        if dinuc in dinucs:
            dinuc_counts[dinucs.index(dinuc)] += 1
    
    dinuc_freqs = sum(count/(seq_len-1) for count in dinuc_counts) if seq_len > 1 else 0

    dataset = [fickett_score, num_islands, gc_count, longest_orf_length, transcript_length, orf_ratio, dinuc_freqs]
    return dataset

In [62]:
def read_fasta(filename): 
    feature_arr = []
    for seq_record in SeqIO.parse(filename, "fasta"):
        rna_seq = Seq(str(seq_record.seq))
        feature = calculate_features(rna_seq)
        feature_arr.append(feature)

    feature_np = np.array(feature_arr)
    feature_pd = pd.DataFrame(data=feature_np, columns=["Ficket score", "CpG Island","GC Content","ORF Length",
                                                   "Transcript Length",  "ORF Ratio","AT Dinucletide"])
    return feature_pd

In [63]:
read_fasta("Micromonas_pusilla_CCMP1545.fasta")

,Ficket score,CpG Island,GC Content,ORF Length,Transcript Length,ORF Ratio,AT Dinucletide
0,2.082236,122.0,68.036530,324.0,657.0,0.493151,0.024390
1,1.251930,148.0,67.352538,357.0,729.0,0.489712,0.031593
2,4.160806,49.0,64.156627,273.0,332.0,0.822289,0.021148
3,1.324757,125.0,68.648649,357.0,740.0,0.482432,0.014885
4,1.097428,55.0,50.840880,282.0,773.0,0.364812,0.046632
...,...,...,...,...,...,...,...
646,1.754686,99.0,71.539961,315.0,513.0,0.614035,0.013672
647,2.118820,42.0,62.585034,225.0,294.0,0.765306,0.030717
648,2.633256,25.0,62.801932,204.0,207.0,0.985507,0.029126
649,1.597523,33.0,64.383562,216.0,219.0,0.986301,0.036697
